# Continuous Group By

### Introduction

In the previous lessons, we worked to create data dashboards, essentially by grouping our data by *discrete* values.  For example, we grouped our sales by the rating, or by the month of the sales.  

But what if we wanted to group our data by continuous variables, like the length of the movie.  This involves a bit more work as we'll explore it in this lesson.

### Grouping by movie length 

Let's use the example of calculating the number of rentals of movies, this time by the length of the movie.  Here, we wouldn't want to group by every minute of movie length, but rather in a range of ten minute increments.  This way we can see how a 100 minute movie performs in comparison with a 110 minute movie, and so on.  

Let's show you the code to accomplish this, and then we'll break it down.

> First we create a connection to our database.

In [7]:
import psycopg2

def get_cursor():
    conn = psycopg2.connect(
    host="127.0.0.1",
    database="pagila_starred",
    user="postgres",
    password="postgres")
    cursor = conn.cursor()
    return cursor

cursor = get_cursor()

And now let's see the query.

In [8]:
query = """
SELECT (dimmovie.length/ 10::integer) as movie_length, COUNT(factsales.sales_key)
FROM factsales 
JOIN dimmovie ON factsales.movie_key = dimmovie.movie_key 
GROUP BY (movie_length)
ORDER BY movie_length DESC LIMIT 8;
"""

In [9]:
cursor.execute(query)
grouped_lengths = cursor.fetchall()
grouped_lengths

[(18, 642),
 (17, 1254),
 (16, 956),
 (15, 1012),
 (14, 1227),
 (13, 1191),
 (12, 1135),
 (11, 1279)]

So above, we have the number of sales of movies of length 180 minutes, 170 minutes and so on.  Take a look at how we accomplished this:

```SQL
SELECT (dimmovie.length/ 10::integer) as movie_length 
```

So we divided the length of the move by 10, (so that 184 minutes now became 18.4, and the coerced this to an integer giving us 18), and then grouped by that coerced number.

In [10]:
grouped_lengths

[(18, 642),
 (17, 1254),
 (16, 956),
 (15, 1012),
 (14, 1227),
 (13, 1191),
 (12, 1135),
 (11, 1279)]

Let's see that query again.

```SQL
SELECT (dimmovie.length/ 10::integer) as movie_length, COUNT(factsales.sales_key)  FROM factsales 
JOIN dimmovie ON factsales.movie_key = dimmovie.movie_key 
GROUP BY (movie_length)
ORDER BY movie_length DESC LIMIT 10;
```

Of course displaying our movie length as 18 minutes when it's really in the range of 180 isn't quite right.  So after rounding down to the integer value, let's multiply our movie value back up by 10.

In [11]:
query = """
SELECT ((dimmovie.length/ 10::integer)* 10)  as movie_length, COUNT(factsales.sales_key)  FROM factsales 
JOIN dimmovie ON factsales.movie_key = dimmovie.movie_key 
GROUP BY (movie_length)
ORDER BY movie_length DESC LIMIT 8;
"""
cursor.execute(query)
cursor.fetchall()

[(180, 642),
 (170, 1254),
 (160, 956),
 (150, 1012),
 (140, 1227),
 (130, 1191),
 (120, 1135),
 (110, 1279)]

So this time we updated the key part of the query above to the following:

```SQL
SELECT ((dimmovie.length/ 10::integer)* 10)  as movie_length
```

So we first divided by 10 to turn into an integer, and then with the decimal gone, we multiplied by 10 again. 

### Practice continuous grouping

Now, below, let try grouping by rental duration.  We want to group in increments of 2 (so how many rentals were 0-2 days, how many 2 -4, etc).

> **Update the code so that it matches the commented out result below**.

In [ ]:
query = """
SELECT (dimmovie.rental_duration) as duration, COUNT(factsales.sales_key)  
FROM factsales
JOIN dimmovie ON factsales.movie_key = dimmovie.movie_key 
GROUP BY (duration)
ORDER BY duration ASC LIMIT 8;
"""
cursor.execute(query)
cursor.fetchall()

# [(2, 3412), (4, 6416), (6, 6221)]

### Summary

In this lesson, we saw how to group by continuous values.  We did so by first dividing by the size of the interval, then rounding to an integer, and then multiplying by the amount we originally divided by.  This is a useful technique with data dashboards.